In [1]:
# print executable path
import sys
print(sys.executable)

/home/leon/tesis/Environments/myenv/bin/python


In [2]:
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.models import Transformer
from torch.utils.data import DataLoader
from sentence_transformers import InputExample

/home/leon/tesis/Environments/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load your custom model (e.g., a Hugging Face model)
my_model = Transformer("./results_300M_diverse_shuffle_75train/checkpoint-5148")

# Wrap it in a SentenceTransformer model
model = SentenceTransformer(modules=[my_model])

Load Data

In [4]:
from datasets import load_dataset

ds = load_dataset("spanish-ir/messirve", "ar")

Generating test split: 100%|██████████| 5481/5481 [00:00<00:00, 131964.34 examples/s]


In [5]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['id', 'query', 'docid', 'docid_text', 'query_date', 'answer_date', 'match_score', 'expanded_search', 'answer_type'],
        num_rows: 22560
    })
    test: Dataset({
        features: ['id', 'query', 'docid', 'docid_text', 'query_date', 'answer_date', 'match_score', 'expanded_search', 'answer_type'],
        num_rows: 5481
    })
})


In [7]:
print(ds["train"][0])

{'id': 4918739, 'query': 'a cual dedo se pone el anillo de compromiso', 'docid': '956254#2', 'docid_text': 'Pero desde hace cientos de años, se dice que la vena amoris pasa por el dedo anular izquierdo que conecta directamente al corazón (téngase en cuenta que la vena amoris no existe realmente). Tradicionalmente, es ofrecido por el hombre como regalo a su novia mientras o cuando ella accede a la proposición de matrimonio. Representa una aceptación formal del futuro compromiso.', 'query_date': datetime.date(2024, 3, 30), 'answer_date': datetime.date(2024, 4, 19), 'match_score': 0.7400000095367432, 'expanded_search': False, 'answer_type': 'feat_snip'}


In [11]:
# Prepare your training data
train_examples = [
    # InputExample(texts=["query", "positive_doc"], label=1.0),
    InputExample(texts=[sample["query"], "positive_doc"], label=sample["match_score"]) for sample in ds["train"]
]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)


In [12]:
# Prepare your training data
test_examples = [
    # InputExample(texts=["query", "positive_doc"], label=1.0),
    InputExample(texts=[sample["query"], "positive_doc"], label=sample["match_score"]) for sample in ds["test"]
]
test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=16)


In [13]:
# Define InfoNCE-like loss
train_loss = losses.MultipleNegativesRankingLoss(model)

# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    output_path="output/mamba-for-messirve",
)

Currently using DataParallel (DP) for multi-gpu training, while DistributedDataParallel (DDP) is recommended for faster training. See https://sbert.net/docs/sentence_transformer/training/distributed.html for more information.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyError: 'sentence_embedding'